## Introduction to our first task: 'Dogs vs Cats'

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [14]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.8.0: cannot open shared 

In [4]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'
batch_size = 64

In [5]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [7]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 79s 220ms/step - loss: 0.2249 - acc: 0.9324 - val_loss: 0.7733 - val_acc: 0.4995
Epoch 2/3
359/359 [==============================] - 75s 208ms/step - loss: 0.1041 - acc: 0.9616 - val_loss: 0.7758 - val_acc: 0.6900
Epoch 3/3
359/359 [==============================] - 75s 210ms/step - loss: 0.0884 - acc: 0.9672 - val_loss: 0.1373 - val_acc: 0.9501
CPU times: user 15min, sys: 35.4 s, total: 15min 36s
Wall time: 3min 49s


In [9]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [10]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, 
                    validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 228s 636ms/step - loss: 0.0885 - acc: 0.9693 - val_loss: 7.9197 - val_acc: 0.5076
CPU times: user 5min, sys: 14.6 s, total: 5min 15s
Wall time: 3min 50s
